# See a Summary of All Activities

In [1]:
from IPython.display import display, Markdown
import snakemd

import fitfile
from garmindb import GarminConnectConfigManager
from garmindb.garmindb import GarminDb, Attributes, ActivitiesDb, Activities, StepsActivities, ActivityLaps, ActivityRecords
from idbutils.list_and_dict import list_not_none

from jupyter_funcs import format_number

gc_config = GarminConnectConfigManager()
db_params_dict = gc_config.get_db_params()


garmin_db = GarminDb(db_params_dict)
garmin_act_db = ActivitiesDb(db_params_dict)
measurement_system = Attributes.measurements_type(garmin_db)
unit_strings = fitfile.units.unit_strings[measurement_system]
distance_units = unit_strings[fitfile.units.UnitTypes.distance_long]

def __report_sport(sport_col, sport):
    records = Activities.row_count(garmin_act_db, sport_col, sport)
    if records > 0:
        sport_title = sport.title().replace('_', ' ')
        total_distance = Activities.get_col_sum_for_value(garmin_act_db, Activities.distance, sport_col, sport)
        if total_distance is None:
            total_distance = 0
            average_distance = 0
        else:
            average_distance = total_distance / records
        return [sport_title, records, format_number(total_distance, 1), format_number(average_distance, 1)]

doc = snakemd.new_doc()

doc.add_heading("Activities Report")
doc.add_paragraph("Analysis of all activities in the database.")

doc.add_table(
    ['Type', 'Count'],
    [
        ["Total activities", Activities.row_count(garmin_act_db)],
        ["Total Lap records", ActivityLaps.row_count(garmin_act_db)],
        ["Activity records", ActivityRecords.row_count(garmin_act_db)],
        ["Fitness activities", Activities.row_count(garmin_act_db, Activities.type, 'fitness')],
        ["Recreation activities", Activities.row_count(garmin_act_db, Activities.type, 'recreation')]
    ])

years = Activities.get_years(garmin_act_db)
years.sort()
doc.add_paragraph(f"Years with activities: {len(years)}: {years}")
sports = list_not_none(Activities.get_col_distinct(garmin_act_db, Activities.sport))
doc.add_paragraph(f"Sports: {', '.join(sports)}")
sub_sports = list_not_none(Activities.get_col_distinct(garmin_act_db, Activities.sub_sport))
doc.add_paragraph(f"SubSports: {', '.join(sub_sports)}")

sports_stats = []
for sport_name in [sport.name for sport in fitfile.Sport]:
    sport_stat = __report_sport(Activities.sport, sport_name)
    if sport_stat:
        sports_stats.append(sport_stat)
doc.add_heading("Sport Type Statistics", 3)
doc.add_table(['Sport', 'Total Activities', f'Total Distance ({distance_units})', f"Average Distance ({distance_units})"], sports_stats)

def __format_activity(activity):
    if activity:
        if activity.is_steps_activity():
            steps_activity = StepsActivities.get(garmin_act_db, activity.activity_id)
            return [activity.activity_id, activity.name, activity.type, activity.sport, format_number(activity.distance, 1), activity.elapsed_time, format_number(activity.avg_speed, 1),
                    steps_activity.avg_pace, format_number(activity.calories), format_number(activity.training_load, 1), activity.self_eval_feel, activity.self_eval_effort]
        return [activity.activity_id, activity.name, activity.type, activity.sport, format_number(activity.distance, 1), activity.elapsed_time, format_number(activity.avg_speed, 1), '',
                format_number(activity.calories), format_number(activity.training_load, 1), activity.self_eval_feel, activity.self_eval_effort]
    return ['', '', '', '', '', '', '', '', '']

activities = Activities.get_latest(garmin_act_db, 10)
rows = [__format_activity(activity) for activity in activities]
doc.add_heading("Last Ten Activities", 3)
doc.add_table(['Id', 'Name', 'Type', 'Sport', f'Distance ({distance_units})', 'Elapsed Time', f'Speed ({unit_strings[fitfile.units.UnitTypes.speed]})',
               f'Pace ({unit_strings[fitfile.units.UnitTypes.pace]})', 'Calories', 'Training Load', 'Feel', 'Effort'], rows)

rows = []
for display_activity in gc_config.display_activities():
    name = display_activity.activity_name().capitalize()
    rows.append([f'Latest {name}'] + __format_activity(Activities.get_latest_by_sport(garmin_act_db, display_activity)))
    rows.append([f'Fastest {name}'] + __format_activity(Activities.get_fastest_by_sport(garmin_act_db, display_activity)))
    rows.append([f'Slowest {name}'] + __format_activity(Activities.get_slowest_by_sport(garmin_act_db, display_activity)))
    rows.append([f'Longest {name}'] + __format_activity(Activities.get_longest_by_sport(garmin_act_db, display_activity)))

doc.add_heading("Interesting Activities", 3)
doc.add_table(['What', 'Id', 'Name', 'Type', 'Sport', f'Distance ({distance_units})', 'Elapsed Time', f'Speed ({unit_strings[fitfile.units.UnitTypes.speed]})',
               f'Pace ({unit_strings[fitfile.units.UnitTypes.pace]})', 'Calories', 'Training Load', 'Feel', 'Effort'], rows)

doc.add_heading("Courses", 3)
courses = Activities.get_col_distinct(garmin_act_db, Activities.course_id)
doc.add_paragraph(str(courses))

display(Markdown(str(doc)))

# Activities Report

Analysis of all activities in the database.

| Type                  | Count   |
| --------------------- | ------- |
| Total activities      | 1026    |
| Total Lap records     | 12394   |
| Activity records      | 3839718 |
| Fitness activities    | 0       |
| Recreation activities | 0       |

Years with activities: 4: [2022, 2023, 2024, 2025]

Sports: cycling, fitness_equipment, running, hiking, generic, swimming, walking, training, paddling, snowmobiling, kayaking

SubSports: generic, strength_training, road, indoor_cycling, mountain, virtual_activity, trail, commuting, lap_swimming, treadmill, open_water, track, pilates, breathing, yoga

### Sport Type Statistics

| Sport             | Total Activities | Total Distance (kilometers) | Average Distance (kilometers) |
| ----------------- | ---------------- | --------------------------- | ----------------------------- |
| Generic           | 12               | 0.0                         | 0.0                           |
| Running           | 156              | 1467.1                      | 9.4                           |
| Cycling           | 630              | 25588.5                     | 40.6                          |
| Fitness Equipment | 151              | 0.0                         | 0.0                           |
| Swimming          | 24               | 59.9                        | 2.5                           |
| Training          | 15               | 0                           | 0                             |
| Walking           | 9                | 23.4                        | 2.6                           |
| Hiking            | 24               | 197.2                       | 8.2                           |
| Paddling          | 3                | 0.0                         | 0.0                           |
| Snowmobiling      | 1                | 5.0                         | 5.0                           |
| Kayaking          | 1                | 13.4                        | 13.4                          |

### Last Ten Activities

| Id          | Name                                            | Type          | Sport   | Distance (kilometers) | Elapsed Time    | Speed (kph) | Pace (per kilometers) | Calories | Training Load | Feel   | Effort         |
| ----------- | ----------------------------------------------- | ------------- | ------- | --------------------- | --------------- | ----------- | --------------------- | -------- | ------------- | ------ | -------------- |
| 21362142436 | Fontainebleau Corrida                           | uncategorized | running | 11.3                  | 01:15:21.062000 | 9.8         | 00:09:49.503370       | 948      | 128.2         | None   | None           |
| 21352449958 | Fontainebleau Corrida                           | uncategorized | running | 10.0                  | 01:08:23.719000 | 10.2        | 00:09:29.075025       | 888      | 131.8         | None   | None           |
| 21338623398 | São Paulo Atletismo                             | uncategorized | running | 6.9                   | 00:53:11.440000 | 8.3         | 00:11:41.545019       | 657      | 182.1         | None   | None           |
| 21321212231 | São Paulo Corrida                               | uncategorized | running | 13.0                  | 01:22:50.588000 | 10.3        | 00:09:25.078716       | 1081     | 125.8         | None   | None           |
| 21305334462 | Porto Alegre Ciclismo de rua                    | uncategorized | cycling | 57.3                  | 02:07:04.835000 | 27.5        |                       | 1508     | 126.3         | None   | None           |
| 21296684866 | Porto Alegre - S2 Sex Treino de ritmo - 300 m v | uncategorized | running | 9.1                   | 00:54:14.836000 | 10.3        | 00:09:24.880387       | 796      | 164.5         | Normal | Extremely Hard |
| 21293387214 | Porto Alegre Corrida                            | uncategorized | running | 3.0                   | 00:18:16.695000 | 10.0        | 00:09:42.251898       | 272      | 60.8          | None   | None           |
| 21288038682 | Porto Alegre Ciclismo de rua                    | uncategorized | cycling | 54.9                  | 01:50:17.688000 | 32.6        |                       | 1193     | 141.5         | None   | None           |
| 21279422134 | Porto Alegre Ciclismo de rua                    | uncategorized | cycling | 44.8                  | 01:48:38.071000 | 27.7        |                       | 1124     | 78.4          | None   | None           |
| 21261701102 | São Paulo - S2 Seg Treino longo - 15 km long    | uncategorized | running | 9.0                   | 00:52:58.749000 | 10.4        | 00:09:19.577278       | 744      | 87.0          | Normal | Hard           |

### Interesting Activities

| What         | Id          | Name                                              | Type          | Sport   | Distance (kilometers) | Elapsed Time    | Speed (kph) | Pace (per kilometers) | Calories | Training Load | Feel | Effort |
| ------------ | ----------- | ------------------------------------------------- | ------------- | ------- | --------------------- | --------------- | ----------- | --------------------- | -------- | ------------- | ---- | ------ |
| Latest Walk  | 17989705267 | São Paulo Caminhada                               | uncategorized | walking | 5.0                   | 01:15:02.132000 | 4.9         | 00:19:42.471917       | 378      | 7.9           | None | None   |
| Fastest Walk | 13302158900 | Caminhada vespertina                              | uncategorized | walking | 3.3                   | 00:10:02        | 20.0        | 00:02:59.985601       | 361      | -             | None | None   |
| Slowest Walk | 12634680406 | Walking                                           | uncategorized | walking | 0.0                   | 01:37:29.429000 | 0.0         | 00:00:00              | 219      | 11.2          | None | None   |
| Longest Walk | 13235433615 | Caminhada vespertina                              | uncategorized | walking | 5.1                   | 01:15:23        | 5.4         | 00:11:12.043011       | 548      | -             | None | None   |
| Latest Run   | 21362142436 | Fontainebleau Corrida                             | uncategorized | running | 11.3                  | 01:15:21.062000 | 9.8         | 00:09:49.503370       | 948      | 128.2         | None | None   |
| Fastest Run  | 20272943357 | Corrida ao entardecer                             | uncategorized | running | 11.1                  | 01:00:40        | 14.9        | 00:04:02.072137       | 1236     | -             | None | None   |
| Slowest Run  | 12828959131 | None                                              | None          | running | 6.4                   | 00:48:25.243000 | -           | 00:00:00              | 641      | -             | None | None   |
| Longest Run  | 12965489701 | Valencia - Valencia Marathon Trinidad Alfonso EDP | race          | running | 42.5                  | 03:46:28.132000 | 11.3        | 00:08:34.331811       | 3245     | 334.6         | None | None   |
| Latest Ride  | 21305334462 | Porto Alegre Ciclismo de rua                      | uncategorized | cycling | 57.3                  | 02:07:04.835000 | 27.5        |                       | 1508     | 126.3         | None | None   |
| Fastest Ride | 11895714656 | Lazy Mountain -1 - TrainerRoad                    | uncategorized | cycling | 28.2                  | 00:40:05        | 42.2        |                       | 230      | 10.8          | None | None   |
| Slowest Ride | 11085371277 | Ciclismo em local fechado                         | uncategorized | cycling | 0.0                   | 00:36:26.905000 | 0.0         |                       | 296      | 14.2          | None | None   |
| Longest Ride | 20999307797 | Bicicleta                                         | uncategorized | cycling | 202.5                 | 08:51:26        | 27.0        |                       | 4392     | 408.6         | None | None   |

### Courses

[None, 173356098, 150507238, 283197208, 195083460, 168020052, 245620244, 133739746, 173664205, 163236213, 294551168, 123840159, 248030939, 234100562, 180628214, 309468054, 178981096, 146196622, 233789699, 423744084, 423509749]